# Solving Sudoku

The primary description of this coursework is available on the CM20252 Moodle page. This is the Jupyter notebook you must complete and submit to receive marks. 

** You must follow all instructions given in this notebook. **

Restart the kernel and run all cells before submitting the notebook. This will guarantee that we will be able to run your code for testing.

Remember to save your work regularly. 

## Getting Started

You already know that you will be writing a Sudoku solver. You need to implement your solver in Python in this notebook. You can use any of the appropriate problem-solving techniques discussed in the lectures. You are encouraged to modify the basic algorithms. Be creative. 

You will be given Sudoku puzzles that either have a single solution or no solution. You will need to identify the solution, if there is one.

Below is a sample puzzle along with its solution. 

<img src="resources/images/Sudoku_unsolved.png" style="width: 200px;"/>
<img src="resources/images/Sudoku_solved.png" style="width: 200px;"/>


## Sample Sudokus

You can test your code on a set of 100 sample Sudoku puzzles. This set is similar to the test set that will be used to assess your work. 

The following code will load 100 Sudoku puzzles and their solutions into two `100x9x9` numpy arrays. Empty cells are designated by zeros. 

In [3]:
import numpy as np

# Load sudokus
sudokus = np.load("resources/data/sudokus.npy")
print("Shape of sudokus array:", sudokus.shape, "; Type of array values:", sudokus.dtype)

# Load solutions
solutions = np.load("resources/data/solutions.npy")
print("Shape of solutions array:", solutions.shape, "; Type of array values:", solutions.dtype, "\n")

sudoku1000 = np.load("resources/data/sudoku-sample-1000.npy")

unsolvable = np.load("resources/data/sudoku-sample-15-unsolvable.npy")
# Print the first sudoku...
print("Sudoku #1:")
print(sudokus[0], "\n")

# ...and its solution
print("Solution of Sudoku #1:")
print(solutions[0])
print("----------")

   

Shape of sudokus array: (100, 9, 9) ; Type of array values: int32
Shape of solutions array: (100, 9, 9) ; Type of array values: int32 

Sudoku #1:
[[0 0 4 3 0 0 2 0 9]
 [0 0 5 0 0 9 0 0 1]
 [0 7 0 0 6 0 0 4 3]
 [0 0 6 0 0 2 0 8 7]
 [1 9 0 0 0 7 4 0 0]
 [0 5 0 0 8 3 0 0 0]
 [6 0 0 0 0 0 1 0 5]
 [0 0 3 5 0 8 6 9 0]
 [0 4 2 9 1 0 3 0 0]] 

Solution of Sudoku #1:
[[8 6 4 3 7 1 2 5 9]
 [3 2 5 8 4 9 7 6 1]
 [9 7 1 2 6 5 8 4 3]
 [4 3 6 1 9 2 5 8 7]
 [1 9 8 6 5 7 4 3 2]
 [2 5 7 4 8 3 9 1 6]
 [6 8 9 7 3 4 1 2 5]
 [7 1 3 5 2 8 6 9 4]
 [5 4 2 9 1 6 3 7 8]]
----------


#### Your code ##

Define a function called `sudoku_solver()` that takes **one** Sudoku puzzle (a $9 \times 9$ numpy array) as input and returns the solved Sudoku as a $9 \times 9$ numpy array. Note that the test set may contain invalid Sudokus, that is, Sudokus with no solution. In such a case, your function should return a $9 \times 9$ numpy array whose values are all equal to `-1`.  

You may use more than one cell to write your code (but this is not necessary).

In [21]:
import numpy as np

def is_valid(sudoku, x, y, n):
    #checks same column and row of (x,y) for n
    for i in range(9):
        if sudoku[x][i] == n and i!=y:
            return False
        elif sudoku[i][y] == n and i!=x:
            return False
    
    for k in range(0,8,3):
        for l in range(0,8,3):
            tmp = np.ndarray.tolist(sudoku[k:k+3,l:l+3])
            blocks.append(tmp)
    for i in blocks:
        blocks2.append(i[0] + i[1] + i[2])

    rowstart = (x//3)*3
    
    colstart = (y//3)

    for i in blocks2[rowstart+colstart]:
        if i == n:
            return False
    return True


# finds the cell in sudoku with least possible values, and fills in cells with only one possible value and then outputs
# cell with least possible values or -1 if it finds a cell that has no possible values - used to find unsolvable sudokus
def infer(sudoku):
    if np.array_equal(sudoku, (-1*np.ones([9,9])).astype(int)):
        return sudoku
    if find_empties(sudoku)[0] == -1:
        return sudoku
    [a,b] = [0,0]
    min_len = 9
    for i in range(9):
        for j in range(9):
            if sudoku[i][j] == 0 and len(possible_vals(sudoku, i , j)) < min_len:
                min_len = len(possible_vals(sudoku, i, j))
                [a,b] = [i, j]
    if min_len == 0:
        return -1*np.ones([9,9]).astype(int)
    elif min_len == 1:
        sudoku[a][b] = possible_vals(sudoku, a, b).pop()
        return infer(sudoku)
    else:
        return sudoku
    
def backtrack(sudoku):
    [row , col, boole] = find_empties(sudoku) 
    
    if not boole:
        return True
    
    
    for i in range(1,10):
        
        if is_valid(sudoku, row, col, i):
            sudoku[row][col] = i
            if backtrack(sudoku) :
                return True
            else:
                sudoku[row][col] = 0
    return False


def find_empties(sudoku):
    #checks if there is any empty cells - use to see if its solved/solvable
    tmp  = False
    #iterates from top to find first empty
    for i in range(9):
        for j in range(9):
            if sudoku[i][j] == 0:
                tmp = True
                return [i , j, tmp] # returns where the empty is and tmp 
    return [-1, -1, tmp] # if this is reached then the sudoku is full and solved


def possible_vals(sudoku, x, y):
    col = set()
    block = set()
    row = set(sudoku[x][:])
    for i in range(9):
        col.add(sudoku[i][y])
    r = (x//3)*3
    c = (y//3)*3
    for i in range(r, r+3):
        for j in range(c, c+3):
            block.add(sudoku[i][j])
    return {1,2,3,4,5,6,7,8,9} - row.union(col, block)

def sudoku_solver(sudoku):
    nope = -1*np.ones([9,9]).astype(int)
    tmp = infer(sudoku)
    if np.array_equal(tmp,nope):
        return nope
    elif backtrack(tmp):
        return sudoku
    else:
        return nope



## Testing your code

You can test your code on the sudoku puzzles that we have provided. For example, the following code cell should return `True` if your `sudoku_solver()` function can correctly solve the first sudoku. This will work only if all of your code is above the test cell. Otherwise, the test cell does not have access to the `sudoku_solver()` function.

In [29]:
sudoku_solver(unsolvable[8])

#print(-1*np.ones([9,9]).astype(int))

array([[-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1]])


## How we will test your code

We will test your code using the hidden tests in the following cell. Specifically, the hidden tests will test your `sudoku_solver()` function on a different set of 100 sudoku puzzles of similar difficulty. **Make sure all of your code is above our test cell. ** Otherwise, the test cell will not have access to the sudoku_solver() function and you will receive zero marks.

In [35]:
# This is the TEST CELL. Do not delete or change. All of your code must be written above this cell. 